In [1]:
import tensorflow as tf
import numpy as np
import pickle
import os

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1  # 为了提高预测准确率，需要对数据进行归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0  # 当前数据集指明遍历的位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)  # 索引进行混排
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """返回batch_size个样本"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
# 建立残差连接块
def residual_block(x, output_channel):
    """残差连接块"""
    input_channel = x.get_shape().as_list()[-1]
    if input_channel * 2 == output_channel:
        increase_dim = True
        strides = (2, 2)
    elif input_channel == output_channel:
        increase_dim = False
        strides = (1, 1)
    else:
        raise Exception("input channel can't match output channel")
    
    conv1 = tf.layers.conv2d(x, output_channel, (3, 3), strides=strides, padding='same', activation=tf.nn.relu, name='conv1')
    
    conv2 = tf.layers.conv2d(conv1, output_channel, (3, 3), strides=(1, 1), padding='same', activation=tf.nn.relu, name='conv2')
    
    if increase_dim:
        pooled_x = tf.layers.average_pooling2d(x, (2, 2), (2, 2), padding='valid')
        
        padded_x = tf.pad(pooled_x, [[0, 0], [0, 0], [0, 0], [input_channel // 2, input_channel // 2]])
    else:
        padded_x = x
    
    output_x = conv2 + padded_x
    return output_x



def res_net(x, num_residual_block, num_filter_base, class_num):
    """
    x: 图片
    num_residual_block: 残差连接块
    num_filter_base: 通道数
    class_num: 
    """
    num_subsampling = len(num_residual_block)
    layers = []
    # x: [None, width, height, channel] -> [width, height, channel]
    input_size = x.get_shape().as_list()[1:]
    with tf.variable_scope('conv0'):
        conv0 = tf.layers.conv2d(x, num_filter_base, (3, 3), strides=(1, 1), padding='same', activation=tf.nn.relu, name='conv0')
        layers.append(conv0)
    for sample_id in range(num_subsampling):
        for i in range(num_residual_block[sample_id]):
            with tf.variable_scope('conv%d_%d' % (sample_id, i)):
                conv = residual_block(layers[-1], num_filter_base*(2**sample_id))
                layers.append(conv)
    multiplier = 2 ** (num_subsampling - 1)
    assert layers[-1].get_shape().as_list()[1:] == [input_size[0] / multiplier, input_size[1] / multiplier, num_filter_base * multiplier]
    with tf.variable_scope('fc'):
        # layer[-1].shape : [None, width, height, channel]
        # kernal_size: image_width, image_height
        global_pool = tf.reduce_mean(layers[-1], [1, 2])
        logits = tf.layers.dense(global_pool, class_num)
        layers.append(logits)
    return layers[-1]



# 计算图模型，先新建计算图，然后将数据塞入计算图中
# placeholder 占位符的作用
x = tf.placeholder(tf.float32, [None, 3072])  # None 样本数不确定，mini_batch
# [None]
y = tf.placeholder(tf.int64, [None])

# 图片转换，reshape
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 大小： 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])



y_ = res_net(x_image, [2, 3, 2], 32, 10)

# 交叉熵损失数函数
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# 执行原理
# y_ -> softmax
# y -> one_hot
# loss -> ylogy_

# 寻找最大概率值的 indices
# 1 代表在第一个维度上
predict = tf.argmax(y_, 1)
# correct_prediction 结果可能是[0,1,0,0,1,1,1]
correct_prediction = tf.equal(predict, y)
# 求准确率，就是求平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

# 在我们有了目标函数和损失函数，再定义梯度下降方法
# 梯度下降的变种
# 优化最小loss的值
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
# 计算图构建完成

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.AveragePooling2D instead.


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

# train 10k: 73.4%
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 100, loss: 2.21734, acc: 0.25000
[Train] Step: 200, loss: 1.97899, acc: 0.25000
[Train] Step: 300, loss: 1.66922, acc: 0.30000
[Train] Step: 400, loss: 1.41754, acc: 0.45000
[Train] Step: 500, loss: 1.96771, acc: 0.25000
[Train] Step: 600, loss: 1.92271, acc: 0.25000
[Train] Step: 700, loss: 1.67884, acc: 0.35000
[Train] Step: 800, loss: 1.33834, acc: 0.70000
[Train] Step: 900, loss: 1.61402, acc: 0.45000
[Train] Step: 1000, loss: 1.70022, acc: 0.35000
(10000, 3072)
(10000,)
[Test ] Step: 1000, acc: 0.44750
[Train] Step: 1100, loss: 1.14684, acc: 0.45000
[Train] Step: 1200, loss: 1.20467, acc: 0.60000
[Train] Step: 1300, loss: 1.40650, acc: 0.50000
[Train] Step: 1400, loss: 1.81084, acc: 0.25000
[Train] Step: 1500, loss: 1.33829, acc: 0.40000
[Train] Step: 1600, loss: 1.15404, acc: 0.65000
[Train] Step: 1700, loss: 1.30136, acc: 0.50000
[Train] Step: 1800, loss: 1.35750, acc: 0.55000
[Train] Step: 1900, loss: 1.32134, acc: 0.65000
[Train] Step: 2000, loss: 1.43940, acc: 0